<div id='id1' />

# Carga de librerias

In [15]:
from elasticsearch import Elasticsearch
from json import dumps
from random import randrange, choice, uniform
import time
from datetime import timedelta, datetime
from random import randint
from lorem_text import lorem
import os
from numpy import asarray, mean

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


<div id='id2' />

# Inserción de documentos individualmente

En esta sección se revisará el tiempo de carga de 10 000 documentos que serán generados aleatoriamente y posteriormente serán cargados de manera individual secuencialmente a la base de datos

## Generación de datos aleatorios

La celda a continuación muestra el script usado para generar los datos aleatorios. Un código similar será incluido en el script será utilizado para generar los documentos JSON que se utilizarán en las siguientes secciones.

In [23]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

d1 = datetime.strptime('1/1/1950 12:59 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2022 01:00 AM', '%m/%d/%Y %I:%M %p')

descripciones = [
    'Transferencia interbancaria SIMPLE',
    'DEB.CTA.P/C.INTERNET',
    'Debito Cta por ACH',
    'Retiro en efectivo',
    'AB.CTA.P/MOVIL',
    'Pago recibo de agua',
    'Debito Cta por compra',
    'Debito Cta por pago de Seguro',
    'Retención IVA',
    'Cargo por cobro de servicio',
    'Cobro seguro',
    'Intereses',
    'Ingreso en efectivo',
    'Reintegro cajero automático',
    'Pago de cheque compensado',
    'Pago recibo de luz',
    'Deposito a cuenta',
    'Comisión de mantenimiento',
    'Pago en interés',
    'Consignación en efectivo',
    'Pago recibo de gas',
    'Compra en comercio',
    'Apertura de cuenta corriente',
    'Pago de nómina',
    'Capitalización de intereses premio',
    'Aporte inversor',
    'Pago de servicio telefónico',
    'Transferencia de fondos de ahorros',
    'Pago en línea - Servicio de internet',
    'Apertura de caja de ahorros'
]

agencias = ['La Paz',
            'Oruro', 
            'Cochabamba', 
            'Chuquisaca', 
            'Pando', 
            'Beni', 
            'Santa Cruz', 
            'Tarija'
]

## Creación de índices

La celda a continuación muestra el script usado para crear los índices de los campos. Un código similar será incluido en el script será utilizado para cargar los documentos a la base de datos en las siguientes secciones.

In [24]:
mapping = {
        "properties": {
            "fecha": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss"
            },
            "agencia": {
                "type": "keyword"
            },
            "monto": {
                "type": "float"
            },
            "descripcion": {
                "type": "text"
            },
            "saldo": {
                "type": "float"
            },
            "nota": {
                "type": "text"
            }
        }
}   

response = es.indices.create(
    index="extracto_cuenta_seq",
    mappings = mapping,
)

# Carga de 10 000 registros de forma individual secuencialmente

En el script a continuación se cargarán 10 000 registros generados aleatoriamente de manera individual y secuencial

In [25]:
start_time = time.time()
time_insert_one = []

for _ in range(10000):
    fecha = str(random_date(d1, d2))
    agencia = choice(agencias)
    monto = round(uniform(10.0, 400000.0), 2)
    descripcion = choice(descripciones)
    saldo = round(uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    one_start_time = time.time()
    es.index(index="extracto_cuenta_seq", document=insert_data)
    one_end_time = time.time()

    time_insert_one.append(one_end_time-one_start_time)

end_time = time.time()

avg_insert_one_time = mean(asarray(time_insert_one))

# Mostramos los resultados en consola

print(f'Tiempo promedio de carga de un solo documento: {str(timedelta(seconds = avg_insert_one_time))}')
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo promedio de carga de un solo documento: 0:00:00.173490
Tiempo de carga de datos: 0:28:55.780249


Según muestran los resultados de ejecución de la celda superior, se tiene un tiempo promedio de carga por documento de **0.17 segundos**, y un tiempo total de carga del total de 10 000 documentos de **28 minutos y 55.78 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_seq``` de la siguiente manera.

In [73]:
cnt = es.cat.count(index="extracto_cuenta_seq", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_seq'")

Actualmente existen 10 documentos en el índice 'extracto_cuenta_seq'


# Inserción de 1 millón de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 1 millón de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 1

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_1M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos ha tomado un tiempo de **4 minutos y 26.16 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_1m``` de la siguiente manera.

In [33]:
cnt = es.cat.count(index="extracto_cuenta_1m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_1m'")

Actualmente existen 1000000 documentos en el índice 'extracto_cuenta_1m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [ ]:
file_sizes = []
n = 1

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data-elastic"))):
    file_stat = os.stat(os.path.join("../json-generated-data-elastic", json_file))
    file_sizes.append(file_stat)

    if i+1 == n:
        break

for file in file_sizes:
    print(f'Documento {json_file} con tamaño en memoria de {file.st_size} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 188495887 bytes


Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [3]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_1m", run_expensive_tasks=True)['extracto_cuenta_1m'], indent=1)
print(r)

{
 "store_size": "138.3mb",
 "store_size_in_bytes": 145089787,
 "all_fields": {
  "total": "137.5mb",
  "total_in_bytes": 144187007,
  "inverted_index": {
   "total": "19mb",
   "total_in_bytes": 19932341
  },
  "stored_fields": "87.7mb",
  "stored_fields_in_bytes": 91994865,
  "doc_values": "12.8mb",
  "doc_values_in_bytes": 13503952,
  "points": "15.9mb",
  "points_in_bytes": 16755849,
  "norms": "1.9mb",
  "norms_in_bytes": 2000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "15.5mb",
   "total_in_bytes": 16300151,
   "inverted_index": {
    "total": "5.1mb",
    "total_in_bytes": 5348000
   },
   "stored_fields": "10.4mb",
   "stored_fields_in_bytes": 10952151,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors": "0b",
   "kn

Extrayendo información importante de los datos recién obtenidos. El tamaño en almacenamiento aproximado de 1 documento de 1 millón de registros con un peso en bruto de 179.76 mb (en escala base binario $2^{20}$), una vez cargado en la base de datos es **77.28 mb** (en escala base binario). Esto nos da un radio de compresión de 0.4299 y una reducción de tamaño en memoria del 57.01%.

Otro número importante al que se le debe prestar atención es el tamaño de los índices en memoria, reportando en el presente caso un tamaño total de todos los índices de **57.26 mb** (en escala base binario). Otro número presente en el reporte de uso de almacenamiento es el número de secuencia, encargado de contar el número de operaciones realizadas en el índice, con un tamaño de **2.95 mb** (en escala base binario). 

Sumando el tamaño de almacenamiento de los documentos, el total de los índices y otros números relevantes, se alcanza un tamaño de almacenamiento en memoria total de **137.51 mb** (en escala base binario).

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_1M_documents.png)

En la figura superior podemos ver que la herramienta gráfica Kibana índica un tamaño de almacenamiento total de **138.37 mb**, número ligeramente mayor al calculado anteriormente, ya que podrían estarse ignorando algunos metadatos pequeños.

| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 4 min 26.16 s  |
| Tamaño JSON bruto                           | 179.76 mb      |
| Tamaño de almacenamiento en elasticsearch   | 77.28 mb       |
| Radio de compresión de documento            | 0.4299         |
| Reducción de tamaño de memoria de documento | 57.01 %        |
| Tamaño total de los índices                 | 57.26 mb       |
| Tamaño de número de secuencia               | 2.95 mb        |
| Tamaño total de almacenamiento colección    | 137.51 mb      |

------------------------

# Inserción de 10 millones de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 10

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_10M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **48 minutos y 52.61 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_10m``` de la siguiente manera.

In [36]:
cnt = es.cat.count(index="extracto_cuenta_10m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_10m'")

Actualmente existen 10000000 documentos en el índice 'extracto_cuenta_10m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [5]:
file_sizes = []
file_names = []
sum = 0

n = 10

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data-elastic"))):
    file_stat = os.stat(os.path.join("../json-generated-data-elastic", json_file))
    file_sizes.append(file_stat)
    file_names.append(json_file)
    sum += int(file_stat.st_size)

    if i+1 == n:
        break

for file, f_name in zip(file_sizes, file_names):
    print(f'Documento {f_name} con tamaño en memoria de {file.st_size} bytes')

print("")
print(f'Sumando un total de {sum} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 188495887 bytes
Documento random-generated-data-10.json con tamaño en memoria de 188447726 bytes
Documento random-generated-data-11.json con tamaño en memoria de 188458209 bytes
Documento random-generated-data-12.json con tamaño en memoria de 188418922 bytes
Documento random-generated-data-13.json con tamaño en memoria de 188411232 bytes
Documento random-generated-data-14.json con tamaño en memoria de 188458816 bytes
Documento random-generated-data-15.json con tamaño en memoria de 188533759 bytes
Documento random-generated-data-16.json con tamaño en memoria de 188525970 bytes
Documento random-generated-data-17.json con tamaño en memoria de 188498006 bytes
Documento random-generated-data-18.json con tamaño en memoria de 188480785 bytes

Sumando un total de 1884729312 bytes


Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_10m``` en la base de datos ejecutaremos el siguiente comando:

In [15]:
r = dumps(es.options(request_timeout=30).indices.disk_usage(index="extracto_cuenta_10m", run_expensive_tasks=True)['extracto_cuenta_10m'], indent=1)
print(r)

{
 "store_size": "1.3gb",
 "store_size_in_bytes": 1440202391,
 "all_fields": {
  "total": "1.3gb",
  "total_in_bytes": 1438808499,
  "inverted_index": {
   "total": "191.3mb",
   "total_in_bytes": 200694994
  },
  "stored_fields": "876.7mb",
  "stored_fields_in_bytes": 919322081,
  "doc_values": "128.7mb",
  "doc_values_in_bytes": 135007131,
  "points": "156.1mb",
  "points_in_bytes": 163784293,
  "norms": "19mb",
  "norms_in_bytes": 20000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "158.1mb",
   "total_in_bytes": 165840919,
   "inverted_index": {
    "total": "53.5mb",
    "total_in_bytes": 56149847
   },
   "stored_fields": "104.6mb",
   "stored_fields_in_bytes": 109691072,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors"

Extrayendo información importante de los datos recién obtenidos. El tamaño en almacenamiento aproximado de 10 documentos de 1 millón de registros con un peso en bruto de 1.75 gb (en escala base binario $2^{30}$), una vez cargado en la base de datos es **0.75 gb** (en escala base binario). Esto nos da un radio de compresión de 0.4296 y una reducción de tamaño en memoria del 57.04%.

Otro número importante al que se le debe prestar atención es el tamaño de los índices en memoria, reportando en el presente caso un tamaño total de todos los índices de **0.56 gb** (en escala base binario). Otro número presente en el reporte de uso de almacenamiento es el número de secuencia, encargado de contar el número de operaciones realizadas en el índice, con un tamaño de **0.028 gb** (en escala base binario). 

Sumando el tamaño de almacenamiento de los documentos, el total de los índices y otros números relevantes, se alcanza un tamaño de almacenamiento en memoria total de **1.34 gb** (en escala base binario).

![Cargando 10 millones de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_10M_documents.png)

| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 48 m  52.61 s  |
| Tamaño JSON bruto                           | 1.75 gb        |
| Tamaño de almacenamiento en elasticsearch   | 0.75 gb        |
| Radio de compresión de documento            | 0.4296         |
| Reducción de tamaño de memoria de documento | 57.04 %        |
| Tamaño total de los índices                 | 0.56 gb        |
| Tamaño de número de secuencia               | 0.028 gb       |
| Tamaño total de almacenamiento colección    | 1.34 gb        |

------------------------

# Inserción de 30 millones de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 30

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_30M_documentos_elastic.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **2 horas 43 minutos y 20.68 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_30m``` de la siguiente manera.

In [16]:
cnt = es.cat.count(index="extracto_cuenta_30m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_30m'")

Actualmente existen 30000000 documentos en el índice 'extracto_cuenta_30m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [18]:
file_sizes = []
file_names = []
sum = 0

n = 30

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data-elastic"))):
    file_stat = os.stat(os.path.join("../json-generated-data-elastic", json_file))
    file_sizes.append(file_stat)
    file_names.append(json_file)
    sum += int(file_stat.st_size)

    if i+1 == n:
        break

for file, f_name in zip(file_sizes, file_names):
    print(f'Documento {f_name} con tamaño en memoria de {file.st_size} bytes')

print("")
print(f'Sumando un total de {sum} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 188495887 bytes
Documento random-generated-data-10.json con tamaño en memoria de 188447726 bytes
Documento random-generated-data-11.json con tamaño en memoria de 188458209 bytes
Documento random-generated-data-12.json con tamaño en memoria de 188418922 bytes
Documento random-generated-data-13.json con tamaño en memoria de 188411232 bytes
Documento random-generated-data-14.json con tamaño en memoria de 188458816 bytes
Documento random-generated-data-15.json con tamaño en memoria de 188533759 bytes
Documento random-generated-data-16.json con tamaño en memoria de 188525970 bytes
Documento random-generated-data-17.json con tamaño en memoria de 188498006 bytes
Documento random-generated-data-18.json con tamaño en memoria de 188480785 bytes
Documento random-generated-data-19.json con tamaño en memoria de 188455315 bytes
Documento random-generated-data-2.json con tamaño en memoria de 188498611 bytes
Documento random-generated-dat

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_30m``` en la base de datos ejecutaremos el siguiente comando:

In [28]:
r = dumps(es.options(request_timeout=40).indices.disk_usage(index="extracto_cuenta_30m", run_expensive_tasks=True)['extracto_cuenta_30m'], indent=1)
print(r)

{
 "store_size": "4gb",
 "store_size_in_bytes": 4312400348,
 "all_fields": {
  "total": "4gb",
  "total_in_bytes": 4310628281,
  "inverted_index": {
   "total": "575.2mb",
   "total_in_bytes": 603210337
  },
  "stored_fields": "2.5gb",
  "stored_fields_in_bytes": 2757840185,
  "doc_values": "386.5mb",
  "doc_values_in_bytes": 405357153,
  "points": "461.7mb",
  "points_in_bytes": 484220606,
  "norms": "57.2mb",
  "norms_in_bytes": 60000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "475.9mb",
   "total_in_bytes": 499018536,
   "inverted_index": {
    "total": "162.2mb",
    "total_in_bytes": 170110453
   },
   "stored_fields": "313.6mb",
   "stored_fields_in_bytes": 328908083,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors":

Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 30 documentos de 1 millón de registros con un peso en bruto de 5.26 gb (en escala binario), una vez cargado en la base de datos es **2.26 gb** (en escala binario). Esto nos da un radio de compresión de 0.4295 y una reducción de tamaño en memoria del 0.5704%.

Otro número importante al que se le debe prestar atención es el tamaño de los índices en memoria, reportando en el presente caso un tamaño total de todos los índices de **1.66 gb** (en escala base binario). Otro número presente en el reporte de uso de almacenamiento es el número de secuencia, encargado de contar el número de operaciones realizadas en el índice, con un tamaño de **0.085 gb** (en escala base binario). 

Sumando el tamaño de almacenamiento de los documentos, el total de los índices y otros números relevantes, se alcanza un tamaño de almacenamiento en memoria total de **4.01 gb** (en escala base binario).

![Cargando 30 millones de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_30M_documents.png)

En la figura superior podemos ver que la herramienta gráfica Kibana índica un tamaño de almacenamiento total de **138.37 mb**, número ligeramente mayor al calculado anteriormente, ya que podrían estarse ignorando algunos metadatos pequeños.

| Tabla resumen de resultados                 | Valor obtenido       |
|---------------------------------------------|----------------------|
| Tiempo de carga                             | 2 h 43 min y 20.68 s |
| Tamaño JSON bruto                           | 5.26 gb              |
| Tamaño de almacenamiento en elasticsearch   | 2.26 gb              |
| Radio de compresión de documento            | 0.4295               |
| Reducción de tamaño de memoria de documento | 57.04 %              |
| Tamaño total de los índices                 | 1.66 gb              |
| Tamaño de número de secuencia               | 0.085 gb             |
| Tamaño total de almacenamiento colección    | 4.01 gb              |

------------------------

# Inserción de 10 millones de documentos (solo 3 campos: fecha, descripción, nota)

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados solo sobre 3 campos: fecha, descripción y nota.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic-3-index.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic-3-index.py -n 10

```

El script antes mencionado recibe como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, se mapeará los campos y declarará cuales se indexarán (fecha, descripción, nota). Posteriormente, se cargará la información almacenada en un documento JSON de forma masiva.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![](../imagenes-de-soporte/elastic-insert/load_10M_documents_3_index.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **47 minutos y 39.02 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_10m_3idx``` de la siguiente manera.

In [29]:
cnt = es.cat.count(index="extracto_cuenta_10m_3idx", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_10m_3idx'")

Actualmente existen 10000000 documentos en el índice 'extracto_cuenta_10m_3idx'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_10m_3idx``` en la base de datos ejecutaremos el siguiente comando:

In [30]:
r = dumps(es.options(request_timeout=30).indices.disk_usage(index="extracto_cuenta_10m_3idx", run_expensive_tasks=True)['extracto_cuenta_10m_3idx'], indent=1)
print(r)

{
 "store_size": "1.2gb",
 "store_size_in_bytes": 1343700157,
 "all_fields": {
  "total": "1.2gb",
  "total_in_bytes": 1343036611,
  "inverted_index": {
   "total": "184.9mb",
   "total_in_bytes": 193949507
  },
  "stored_fields": "876.7mb",
  "stored_fields_in_bytes": 919336222,
  "doc_values": "128.7mb",
  "doc_values_in_bytes": 135012715,
  "points": "71.2mb",
  "points_in_bytes": 74738167,
  "norms": "19mb",
  "norms_in_bytes": 20000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "158.2mb",
   "total_in_bytes": 165982774,
   "inverted_index": {
    "total": "53.8mb",
    "total_in_bytes": 56453555
   },
   "stored_fields": "104.4mb",
   "stored_fields_in_bytes": 109529219,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors": 

Por la información recopilada anteriormente, se sabe que el tamaño en almacenamiento aproximado de 10 documentos de 1 millón de registros con un peso en bruto de 1.75 gb (en escala base binario), una vez cargado en la base de datos es **0.75 gb** (en escala base binario). Esto nos da un radio de compresión de 0.4297 y una reducción de tamaño en memoria del 57.03%.

Otro número importante al que se le debe prestar atención es el tamaño de los índices en memoria, reportando en el presente caso un tamaño total de todos los índices de **0.47 gb** (en escala base binario). Otro número presente en el reporte de uso de almacenamiento es el número de secuencia, encargado de contar el número de operaciones realizadas en el índice, con un tamaño de **0.028 gb** (en escala base binario). 

Sumando el tamaño de almacenamiento de los documentos, el total de los índices y otros números relevantes, se alcanza un tamaño de almacenamiento en memoria total de **1.25 gb** (en escala base binario).

![Cargando 10 millones de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_10M_documents_3idx.png)

| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 47 min 39.02 s |
| Tamaño JSON bruto                           | 1.75 gb        |
| Tamaño de almacenamiento en elasticsearch   | 0.75 gb        |
| Radio de compresión de documento            | 0.4297         |
| Reducción de tamaño de memoria de documento | 57.03 %        |
| Tamaño total de los índices                 | 0.47 gb        |
| Tamaño de número de secuencia               | 0.028 gb       |
| Tamaño total de almacenamiento colección    | 1.25 gb        |

------------------------

# Inserción de 30 millones de documentos (solo 3 campos: fecha, descripción, nota)

En esta sección se revisará la carga y el tamaño de almacenamiento de 30 millones de documentos indexados solo sobre 3 campos: fecha, descripción y nota.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic-3-index.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic-3-index.py -n 30

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![](../imagenes-de-soporte/elastic-insert/load_30M_documentos_elastic_3_index.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **2 horas 17 minutos y 2.55 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_30m_3idx``` de la siguiente manera.

In [32]:
cnt = es.cat.count(index="extracto_cuenta_30m_3idx", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_30m_3idx'")

Actualmente existen 30000000 documentos en el índice 'extracto_cuenta_30m_3idx'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_30m``` en la base de datos ejecutaremos el siguiente comando:

In [39]:
r = dumps(es.options(request_timeout=30).indices.disk_usage(index="extracto_cuenta_30m_3idx", run_expensive_tasks=True)['extracto_cuenta_30m_3idx'], indent=1)
print(r)

{
 "store_size": "3.7gb",
 "store_size_in_bytes": 4045396917,
 "all_fields": {
  "total": "3.7gb",
  "total_in_bytes": 4043737642,
  "inverted_index": {
   "total": "555.3mb",
   "total_in_bytes": 582330007
  },
  "stored_fields": "2.5gb",
  "stored_fields_in_bytes": 2757844242,
  "doc_values": "386.5mb",
  "doc_values_in_bytes": 405330672,
  "points": "227.1mb",
  "points_in_bytes": 238232721,
  "norms": "57.2mb",
  "norms_in_bytes": 60000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "475.1mb",
   "total_in_bytes": 498219743,
   "inverted_index": {
    "total": "162.1mb",
    "total_in_bytes": 170057039
   },
   "stored_fields": "312.9mb",
   "stored_fields_in_bytes": 328162704,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vecto

Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 30 documentos de 1 millón de registros con un peso en bruto de 5.26 gb (en escala binario), una vez cargado en la base de datos es **2.26 gb** (en escala binario). Esto nos da un radio de compresión de 0.4297 y una reducción de tamaño en memoria del 57.03%.

Otro número importante al que se le debe prestar atención es el tamaño de los índices en memoria, reportando en el presente caso un tamaño total de todos los índices de **1.42 gb** (en escala base binario). Otro número presente en el reporte de uso de almacenamiento es el número de secuencia, encargado de contar el número de operaciones realizadas en el índice, con un tamaño de **0.086 gb** (en escala base binario). 

Sumando el tamaño de almacenamiento de los documentos, el total de los índices y otros números relevantes, se alcanza un tamaño de almacenamiento en memoria total de **3.77 gb** (en escala base binario).

![](../imagenes-de-soporte/elastic-insert/storage_kibana_30M_documents_3_idx.png)

| Tabla resumen de resultados                 | Valor obtenido       |
|---------------------------------------------|----------------------|
| Tiempo de carga                             | 2 h 17 min  2.55 s   |
| Tamaño JSON bruto                           | 5.26 gb              |
| Tamaño de almacenamiento en elasticsearch   | 2.26 gb              |
| Radio de compresión de documento            | 0.4297               |
| Reducción de tamaño de memoria de documento | 57.03 %              |
| Tamaño total de los índices                 | 1.42 gb              |
| Tamaño de número de secuencia               | 0.086 gb             |
| Tamaño total de almacenamiento colección    | 3.77 gb              |